In [3]:
# https://www.kaggle.com/alexanderkireev/experiments-with-imbalance-nn-architecture/code


import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'
import gc


In [4]:
train_df = pd.read_csv('train_data_code2.csv')
test_df = pd.read_csv('test_data_code2.csv')
y_train = pd.read_csv('y_code2.csv')

print(train_df.shape)
print(y_train.shape)


(184903890, 11)
(184903890, 2)


In [14]:
print(y_train.head())
print(y_train.values)
yy = y_train.values
print(yy[:,1])
y_train = yy[:,1]
del yy; gc.collect()



AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [9]:

print ('neural network....')
from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
from keras.callbacks import Callback
from keras.models import Sequential
from keras.optimizers import Adam

neural network....


Using TensorFlow backend.


In [10]:
# simple MY nn

my_init = 'random_uniform'
model = Sequential()
model.add(Dense(32,activation='relu',kernel_initializer=my_init,
                bias_initializer='zeros',input_shape=(train_df.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(Dense(64,kernel_initializer=my_init,
                bias_initializer='zeros',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# model.add(Dense(128,kernel_initializer=my_init,
#                 bias_initializer='zeros',activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(64,kernel_initializer=my_init,
#                 bias_initializer='zeros',activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))

model.add(Dense(32,kernel_initializer=my_init,
                bias_initializer='zeros',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(16,kernel_initializer=my_init,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(1,activation='sigmoid'))


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                384       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
__________

In [16]:

epochs = 3
batch_size = 10000
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(list(train_df)[0]) / batch_size) * epochs
lr_init, lr_fin = 0.002, 0.0002
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.002, decay=lr_decay)
model.compile(loss='binary_crossentropy',optimizer=optimizer_adam,metrics=['accuracy'])
class_weight = {0:.01,1:.99}
model.fit(train_df, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weight, shuffle=True, verbose=2)
del train_df, y_train; gc.collect()

json_string = model.to_json()
with open("imbalanced_model.json", "w") as json_file:
    json_file.write(json_string)
model.save_weights('imbalanced_data.h5')

Epoch 1/3
 - 11079s - loss: 0.0062 - acc: 0.9964
Epoch 2/3
 - 12407s - loss: 0.0062 - acc: 0.9975
Epoch 3/3
 - 11768s - loss: 0.0062 - acc: 0.9975


In [18]:
def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'd': np.array(dataset.day),
        'wd': np.array(dataset.wday),
        'qty': np.array(dataset.qty),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X

In [26]:
# print(test_df.head())

print(a)

          click_id
0                0
1                1
2                2
3                3
4                4
5                5
6                6
7                7
8                9
9                8
10              10
11              11
12              12
13              13
14              14
15              15
16              16
17              20
18              19
19              17
20              18
21              21
22              22
23              23
24              24
25              25
26              26
27              27
28              28
29              29
...            ...
18790439  18790438
18790440  18790440
18790441  18790441
18790442  18790442
18790443  18790443
18790444  18790444
18790445  18790445
18790446  18790446
18790447  18790447
18790448  18790448
18790449  18790449
18790450  18790450
18790451  18790451
18790452  18790452
18790453  18790453
18790454  18790455
18790455  18790454
18790456  18790456
18790457  18790457
18790458  18790458
18790459  18

In [32]:


sub = pd.DataFrame()
sub['click_id'] = a['click_id']

test_df.drop(['click_time','ip','is_attributed'],1,inplace=True)
test_df = get_keras_data(test_df)


AttributeError: 'dict' object has no attribute 'head'

In [30]:
print("predicting....")
sub['is_attributed'] = model.predict(test_df, batch_size=batch_size, verbose=2)
del test_df; gc.collect()
print("writing....")
sub.to_csv('imbalanced_data.csv',index=False)

predicting....


ValueError: No data provided for "dense_1_input". Need data for each key in: ['dense_1_input']